In [131]:
import pandas as pd
import numpy as np
import nltk
import os
import matplotlib.pyplot as plt
import seaborn as sns
import sys
from nltk.corpus import treebank
import random

import spacy
import spacy_transformers

from sklearn import svm
from sklearn.model_selection import train_test_split, cross_val_score
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.metrics import accuracy_score, classification_report

import swifter  # to parallelize pandas apply, but actually useless here bc doesnt work with strings

In [19]:
filename = '/Users/christian_g/PycharmProjects/Data_Science_Projects/data/nlp/indeed_resumes.csv'
df = pd.read_csv(filename)

In [20]:
df.rename(mapper={'class':'classification'}, axis=1, inplace=True)
df.head()

,resume_id,classification,resume_text
0,resume_1,not_flagged,\rCustomer Service Supervisor/Tier - Isabella ...
1,resume_2,not_flagged,\rEngineer / Scientist - IBM Microelectronics ...
2,resume_3,not_flagged,\rLTS Software Engineer Computational Lithogra...
3,resume_4,not_flagged,TUTOR\rWilliston VT - Email me on Indeed: ind...
4,resume_5,flagged,\rIndependent Consultant - Self-employed\rBurl...


In [35]:
df.classification.value_counts()

classification
not_flagged    92
flagged        33
Name: count, dtype: int64

# Bag of Words SVC

In [62]:
# vectorizing responses
vectorizer = CountVectorizer(ngram_range = (1, 2))
x = vectorizer.fit_transform(df.resume_text).toarray()
len(x), len(x_vectors[0])

(125, 68932)

In [63]:
y = list(df.classification)

In [64]:
x_train, x_test, y_train, y_test = train_test_split(x, y, test_size=0.2)

In [65]:
model = svm.SVC(kernel='linear')

In [66]:
model.fit(x_train, y_train)

SVC(kernel='linear')

In [67]:
y_out = model.predict(x_test)
accuracy_score(y_out, y_test)

0.72

In [68]:
print(classification_report(y_out, y_test))

              precision    recall  f1-score   support

     flagged       0.29      0.50      0.36         4
 not_flagged       0.89      0.76      0.82        21

    accuracy                           0.72        25
   macro avg       0.59      0.63      0.59        25
weighted avg       0.79      0.72      0.75        25



# Word Vectors SVC

In [69]:
# loading word vectors
nlp = spacy.load('en_core_web_lg')

In [73]:
docs = df['resume_text'].apply(lambda x: nlp(x))

In [75]:
x = [doc.vector for doc in docs]

In [78]:
x_train, x_test, y_train, y_test = train_test_split(x, y, test_size=0.2)

In [79]:
model = svm.SVC(kernel='linear')

In [80]:
model.fit(x_train, y_train)

SVC(kernel='linear')

In [81]:
y_out = model.predict(x_test)
accuracy_score(y_out, y_test)

0.72

In [82]:
print(classification_report(y_out, y_test))

              precision    recall  f1-score   support

     flagged       0.44      0.67      0.53         6
 not_flagged       0.88      0.74      0.80        19

    accuracy                           0.72        25
   macro avg       0.66      0.70      0.67        25
weighted avg       0.77      0.72      0.74        25



# Transformer SVC

In [107]:
nlp = spacy.load('en_core_web_trf')

In [108]:
docs = df['resume_text'].apply(lambda x: nlp(x))

In [109]:
docs[23]._.trf_data.all_outputs[0].data

array([[ 0.18166955, -0.21086842,  0.3854337 , ...,  0.10049613,
        -0.76851976, -1.5271969 ],
       [-0.8064627 , -0.05206586,  0.8816481 , ..., -0.40782687,
        -0.67794627, -0.4019547 ],
       [ 0.        ,  0.        ,  0.        , ...,  0.        ,
         0.        ,  0.        ],
       ...,
       [-0.39214694, -0.4402416 , -0.04586587, ...,  0.681843  ,
        -0.17380245, -0.33407754],
       [-0.46333456, -0.9720196 , -0.35388893, ...,  0.8591167 ,
         0.11116373, -0.2054527 ],
       [ 0.        ,  0.        ,  0.        , ...,  0.        ,
         0.        ,  0.        ]], dtype=float32)

In [160]:
# this is probably not even the right way to do this but I have no idea wtf I'm doing
x = [doc._.trf_data.all_outputs[0].data[-1] for doc in docs]

In [125]:
x_train, x_test, y_train, y_test = train_test_split(x, y, test_size=0.2)

In [126]:
model = svm.SVC(kernel='linear')

In [127]:
model.fit(x_train, y_train)

SVC(kernel='linear')

In [130]:
y_out = model.predict(x_test)
accuracy_score(y_out, y_test)

0.76

# Cross Validation for Transformers SVC

In [157]:
model = svm.SVC(kernel='linear')

In [161]:
# fit SVMs for 5 distinct 20% test splits, get accuracy scores
scores = cross_val_score(model, x, y, cv=5)
scores

array([0.76, 0.76, 0.72, 0.72, 0.72])

In [159]:
print(f'Average accuracy: {scores.mean()} \nStdev: {scores.std()}')

Average accuracy: 0.736 
Stdev: 0.019595917942265444
